## 데이터 웹 크롤링 코드

In [ ]:
import os
import time
import shutil
from PIL import Image, ImageStat
from icrawler.builtin import GoogleImageCrawler
import imagehash
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

resize_size = (512, 512)

# --- 이미지 품질 및 필터 함수 ---
def is_blurry(image, threshold=100):
    try:
        img = np.array(image)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        lap = cv2.Laplacian(gray, cv2.CV_64F).var()
        return lap < threshold
    except:
        return False

def is_mostly_black(image, threshold=15):
    stat = ImageStat.Stat(image)
    avg_brightness = sum(stat.mean) / len(stat.mean)
    return avg_brightness < threshold

def resize_images_in_folder(folder_path, size):
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            with Image.open(fpath).convert("RGB") as img:
                if is_blurry(img) or is_mostly_black(img):
                    os.remove(fpath)
                    continue
                img = img.resize(size)
                new_fpath = os.path.splitext(fpath)[0] + ".jpg"
                img.save(new_fpath, "JPEG")
                if fpath != new_fpath:
                    os.remove(fpath)
        except Exception as e:
            print(f"⚠️ 리사이징 실패: {fname} → {e}")
            os.remove(fpath)

def remove_duplicate_images(folder_path, hash_size=16):
    seen_hashes = set()
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            with Image.open(fpath) as img:
                img_hash = imagehash.phash(img, hash_size=hash_size)
            if img_hash in seen_hashes:
                os.remove(fpath)
                print(f"🗑️ 중복 제거: {fname}")
            else:
                seen_hashes.add(img_hash)
        except Exception as e:
            print(f"⚠️ 해시 실패: {fname} → {e}")
            os.remove(fpath)

# --- 폴더 내 이미지의 해시 값들을 계산 ---
def compute_hashes_in_folder(folder_path, hash_size=16):
    hashes = set()
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            with Image.open(fpath) as img:
                h = imagehash.phash(img, hash_size=hash_size)
                hashes.add(str(h))
        except Exception as e:
            print(f"⚠️ 해시 계산 실패: {fname} → {e}")
    return hashes

# ─── 부족한 클래스에 대해 재크롤링 (임시 폴더에 크롤링 후 중복 체크) ─────────────────────────
def crawl_new_images_for_class(class_name, search_queries, main_folder, temp_folder, max_num=50, hash_size=16):
    os.makedirs(temp_folder, exist_ok=True)
    # 기존 이미지들의 해시 계산
    existing_hashes = compute_hashes_in_folder(main_folder, hash_size=hash_size)
    
    for query in search_queries:
        query_folder = os.path.join(temp_folder, query.replace(" ", "_"))
        os.makedirs(query_folder, exist_ok=True)
        print(f"[{class_name}] 새로운 이미지 크롤링 중 - 검색어: '{query}'")
        crawler = GoogleImageCrawler(storage={"root_dir": query_folder})
        crawler.crawl(
            keyword=query,
            max_num=max_num,
            filters={"type": "photo", "size": "large"}
        )
        
        resize_images_in_folder(query_folder, resize_size)
        
        for fname in os.listdir(query_folder):
            fpath = os.path.join(query_folder, fname)
            try:
                with Image.open(fpath) as img:
                    h = str(imagehash.phash(img, hash_size=hash_size))
                if h in existing_hashes:
                    os.remove(fpath)
                    print(f"중복 이미지 제거됨: {fname}")
                else:
                    dest_path = os.path.join(main_folder, fname)
                    shutil.move(fpath, dest_path)
                    existing_hashes.add(h)
            except Exception as e:
                print(f"이미지 처리 오류 {fname}: {e}")
        
        if not os.listdir(query_folder):
            os.rmdir(query_folder)
        time.sleep(5)

    remove_duplicate_images(main_folder, hash_size=hash_size)

# ─── 크롤 대상 클래스 (검색어 리스트) ──────────────────────────────────────────────
crawl_targets = {
    "wood": ["wood", "plywood pieces", "furniture wood pieces"],
    "food_waste": [
        "egg shells", "duck egg shells", "quail egg shells", "ostrich egg shells",
        "walnut shells", "peanut shells", "chestnut shells", "acorn shells",
        "pineapple peels", "corn husks", "corn cobs",
        "green tea leaves", "herbal medicine residue",
        "pork bones", "beef bones", "chicken bones", "fish bones",
        "clam shells", "crab shells", "lobster shells",
        "rice husks", "food waste", "onion peels"
    ],
    "general_waste": ["ceramic dishes", "porcelain items"],
    "paper": ["paper packs", "paper cups", "newspapers", "books", "notebooks", "cardboard boxes"],
    "glass": ["glass bottles", "broken glass"],
    "can": ["steel cans", "aluminum cans", "butane gas cans", "pesticide cans"],
    "plastic": ["clear PET bottles", "colored PET bottles", "plastic bags"],
    "styrofoam": ["styrofoam", "contaminated styrofoam"],
    "battery": ["batteries", "AA batteries", "AAA batteries"],
    "electronics": ["TVs", "refrigerators", "washing machines", "air conditioners", "computers", "mobile phones"],
    "lighting": ["fluorescent lamps"],
    "metal": ["scrap metal", "iron pipes"],
    "clothing": ["clothes", "old clothes"]
}

# 각 검색어 문자열의 끝에 " waste img"를 추가한 새로운 딕셔너리 생성
modified_crawl_targets = {
    category: [f"{query} waste img" for query in queries]
    for category, queries in crawl_targets.items()
}

print(modified_crawl_targets)

base_dir = r"C:\Users\Administrator\Downloads\end-to-end-image-scraper\downloaded_images"

# ─── 각 카테고리별 이미지 크롤링 함수 (진행 바 포함) ──────────────────────────────
def crawl_images_by_category(crawl_targets, base_dir, max_per_variant=100):
    for category, queries in crawl_targets.items():
        category_folder = os.path.join(base_dir, category)
        os.makedirs(category_folder, exist_ok=True)
        print(f"\n=== 카테고리: {category} ===")
        # tqdm을 사용하여 각 검색어 진행 상황을 표시합니다.
        for query in tqdm(queries, desc=f"Processing {category}", unit="query"):
            sub_folder = os.path.join(category_folder, query.replace(" ", "_"))
            os.makedirs(sub_folder, exist_ok=True)
            print(f"📦 크롤링: '{query}' → {sub_folder}")
            crawler = GoogleImageCrawler(storage={"root_dir": sub_folder})
            crawler.crawl(
                keyword=query,
                max_num=max_per_variant,
                filters={"type": "photo", "size": "large"}
            )
            resize_images_in_folder(sub_folder, resize_size)
            remove_duplicate_images(sub_folder)
            time.sleep(10)

# ─── 함수: 클래스별 이미지 수 ──────────────────────────────
def count_images_per_class(base_dir):
    counts = {}
    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        total = 0
        for root, _, files in os.walk(category_path):
            total += len([f for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))])
        counts[category] = total
    return counts

# ─── 메인 실행 부분 ──────────────────────────────
crawl_images_by_category(modified_crawl_targets, base_dir, max_per_variant=300)

# ─── 전체 실행 후, 각 카테고리별 이미지 수를 시각화 (막대그래프) ──────────────────────────────
def visualize_image_counts(base_dir):
    counts = count_images_per_class(base_dir)
    categories = list(counts.keys())
    image_counts = list(counts.values())

    plt.figure(figsize=(12, 6))
    plt.bar(categories, image_counts, color='skyblue')
    plt.xlabel("카테고리")
    plt.ylabel("이미지 개수")
    plt.title("카테고리별 이미지 수")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

visualize_image_counts(base_dir)


## 부족한 부분 재크롤링 코드

In [ ]:
import os
import time
from PIL import Image, ImageStat
import imagehash
import numpy as np
import cv2
from icrawler.builtin import GoogleImageCrawler

# 기존에 정의된 함수들 (resize_images_in_folder, is_blurry, is_mostly_black, remove_duplicate_images, crawl_trash_images)
# 는 이미 포함되어 있다고 가정합니다.
# ---------------------------------------------------
# 아래는 재크롤링 및 데이터 정제 흐름에 필요한 추가 함수들입니다.

# 1. 기존 이미지들의 해시값 수집
def get_existing_hashes(base_dir):
    existing_hashes = set()
    for root, _, files in os.walk(base_dir):
        for fname in files:
            fpath = os.path.join(root, fname)
            try:
                with Image.open(fpath) as img:
                    h = imagehash.phash(img)
                    existing_hashes.add(h)
            except Exception as e:
                continue
    return existing_hashes

# 2. 기존 해시와 비교하여 중복 제거 (이미 수집된 해시 set을 업데이트하며 진행)
def remove_duplicate_with_existing(folder_path, existing_hashes, hash_size=16):
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            with Image.open(fpath) as img:
                img_hash = imagehash.phash(img, hash_size=hash_size)
            if img_hash in existing_hashes:
                os.remove(fpath)
                print(f"🗑️ 기존 중복 제거: {fname}")
            else:
                existing_hashes.add(img_hash)
        except Exception as e:
            print(f"⚠️ 해시 실패: {fname} → {e}")
            os.remove(fpath)

# 3. 각 클래스(카테고리)별 이미지 수 확인 함수
def get_class_counts(base_dir):
    counts = {}
    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            num_images = len([
                f for f in os.listdir(category_path)
                if os.path.isfile(os.path.join(category_path, f))
            ])
            counts[category] = num_images
    return counts

# 4. 부족한 클래스 재크롤링을 위한 함수 (추가 modifier 활용)
def recrawl_category_with_modifier(category, modifier, save_path, max_per_variant=50):
    search_query = f"{modifier} {category}"
    sub_folder = f"{category.replace(' ', '_')}_{modifier.replace(' ', '_')}"
    output_path = os.path.join(save_path, sub_folder)
    os.makedirs(output_path, exist_ok=True)
    
    print(f"📦 재크롤링: {search_query} → {output_path}")
    crawler = GoogleImageCrawler(storage={"root_dir": output_path})
    crawler.crawl(
        keyword=search_query,
        max_num=max_per_variant,
        filters={"type": "photo", "size": "large"}
    )
    
    resize_images_in_folder(output_path, resize_size)
    # 재크롤링 시 기존 해시와 비교하여 중복 이미지 제거
    remove_duplicate_with_existing(output_path, existing_hashes)
    time.sleep(10)

# ---------------------------------------------------
# 이미 진행한 초기 크롤링 코드 (예시)
for category in crawl_categories:
    category_path = os.path.join(base_dir, category.replace(" ", "_"))
    crawl_trash_images(category, category_path, max_per_variant=100)

# ---------------------------------------------------
# ① 기존 데이터셋의 모든 이미지 해시 수집
existing_hashes = get_existing_hashes(base_dir)

# ② 각 카테고리 폴더에서 기존 해시와 비교하여 중복 이미지 제거
for category in crawl_categories:
    category_folder = os.path.join(base_dir, category.replace(" ", "_"))
    if os.path.exists(category_folder):
        remove_duplicate_with_existing(category_folder, existing_hashes)

# ③ 클래스별 이미지 수 확인
class_counts = get_class_counts(base_dir)
print("현재 클래스별 이미지 수:")
for cat, count in class_counts.items():
    print(f"{cat}: {count}")

# ④ 부족한 클래스에 대해 재크롤링 수행 (예: 최소 200장 이상 확보)
minimum_images = 200
# 추가 크롤링을 위한 extra modifier 리스트
extra_modifiers = ["recycled", "old", "broken", "dirty", "used", "disposed"]

# 각 카테고리별 부족한 이미지 수에 대해 재크롤링 진행
class_counts = get_class_counts(base_dir)
for category, count in class_counts.items():
    if count < minimum_images:
        print(f"카테고리 '{category}'의 이미지가 부족합니다 ({count}개). 재크롤링을 시작합니다.")
        category_folder = os.path.join(base_dir, category)
        # 각 추가 modifier로 재크롤링 시도
        for modifier in extra_modifiers:
            recrawl_category_with_modifier(category, modifier, category_folder, max_per_variant=50)

# ⑤ 최종 클래스별 이미지 수 확인
final_counts = get_class_counts(base_dir)
print("최종 클래스별 이미지 수:")
for cat, count in final_counts.items():
    print(f"{cat}: {count}")


## 이미지 데이터셋 예측 후 분류를 위한 코드 (정확도 떨어짐)

import os

# 데이터셋 경로 지정 (r""를 사용하여 백슬래시를 이스케이프 처리)
dataset_path = r"C:\Users\Administrator\Downloads\capstone data\DATASET"

# 폴더 내의 파일 목록 가져오기
files = os.listdir(dataset_path)

# 이미지 파일 확장자 리스트 (필요에 따라 추가)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

# 이미지 파일만 필터링
image_files = [f for f in files if os.path.splitext(f)[1].lower() in image_extensions]

print("총 이미지 개수:", len(image_files))


import os
import torch
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import json
import urllib.request

# 사전 학습된 모델(ResNet50) 로드 및 평가 모드 전환
model = models.resnet50(pretrained=True)
model.eval()

# 이미지 전처리 파이프라인 구성
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ImageNet 클래스 정보를 가져오기 위해 카테고리 맵을 다운로드 (예시)
url, filename = ("https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json", "imagenet_class_index.json")
urllib.request.urlretrieve(url, filename)
with open('imagenet_class_index.json') as f:
    class_idx = json.load(f)
idx2label = {int(key): value[1] for key, value in class_idx.items()}

# 이미지가 저장된 폴더 경로 설정
image_dir = r"C:\Users\Administrator\Downloads\capstone data\DATASET\TRAIN\R"
results = []

# 폴더 내 각 이미지에 대해 예측 실행
for image_file in os.listdir(image_dir):
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(image_dir, image_file)
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0)
        with torch.no_grad():
            output = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        confidence, predicted_idx = torch.max(probabilities, dim=0)
        predicted_label = idx2label[predicted_idx.item()]
        results.append({
            "filename": image_file,
            "predicted_label": predicted_label,
            "confidence": confidence.item()
        })


# 예측 결과를 CSV로 저장
df = pd.DataFrame(results)
df.to_csv("predictions.csv", index=False)

# 예측 결과를 JSON으로 저장
with open("predictions.json", "w") as f:
    json.dump(results, f, indent=4)

## 예측 후 label studio 연결 시도

from flask import Flask, request, jsonify
import json
import os

app = Flask(__name__)

# 미리 저장된 예측 결과 파일 불러오기
# predictions.json 파일은 아래와 같이 구성했다고 가정합니다.
# [
#   {"filename": "image1.jpg", "predicted_label": "wood", "confidence": 0.95},
#   {"filename": "image2.jpg", "predicted_label": "plastic", "confidence": 0.90},
#   ...
# ]
with open('predictions.json', 'r') as f:
    predictions = json.load(f)

@app.route('/predict', methods=['POST'])
def predict():
    # Label Studio에서 보내는 요청의 JSON 내용
    input_data = request.get_json()
    print("Received request:", input_data)
    
    # Label Studio 요청 JSON에서 이미지 URL(또는 경로)를 추출합니다.
    # 일반적으로는 task의 data 항목에 image URL이 포함되어 있습니다.
    image_url = input_data.get('data', {}).get('image', '')
    
    # 파일명 추출: 이미지 URL이 http://.../image1.jpg 같은 형태라면 마지막 부분을 사용합니다.
    filename = os.path.basename(image_url)
    
    # 예측 결과 탐색: predictions.json에 저장된 결과에서 파일명으로 매칭
    result_payload = {}
    for pred in predictions:
        if pred.get("filename") == filename:
            # Label Studio가 이해할 수 있는 예측 포맷 생성
            result_payload = {
                "result": [
                    {
                        "from_name": "label",       # Label Studio에서 지정한 라벨 위젯의 이름
                        "to_name": "image",         # 라벨링 대상(예: 이미지)의 이름
                        "type": "choices",          # task 종류 (여기서는 단일 선택 분류)
                        "value": {
                            "choices": [pred.get("predicted_label")]
                        }
                    }
                ]
            }
            break

    # 예측 결과가 없는 경우 빈 결과 반환
    if not result_payload:
        result_payload = {"result": []}
    
    print("Returning prediction:", result_payload)
    return jsonify(result_payload)

if __name__ == '__main__':
    # 서버를 0.0.0.0:5000에서 실행하여 Label Studio가 접근할 수 있도록 설정
    app.run(host='0.0.0.0', port=5000)


## 파일 분류 및 이동 코드

In [ ]:
import os, re, shutil

# 기존에 사용한 함수 (파일명에서 숫자를 추출하는 함수)
def extract_idx(fname):
    """
    파일명에서 R_123 또는 R123 형태로 된 숫자를 뽑아 정수로 반환.
    매치되지 않으면 None 반환.
    """
    base = os.path.splitext(fname)[0]  # ex. "R_1" 또는 "R1"
    m = re.match(r"^R_?(\d+)$", base)   # R 또는 R_ 뒤 숫자 캡처
    return int(m.group(1)) if m else None

# 기본 경로 설정 (필요에 따라 변경)
dst_root = r"C:\Users\Administrator\Downloads\capstone data\DATASET\TRAIN"

# paper 폴더 경로 (기존 분류 작업으로 생성된 폴더)
paper_folder = os.path.join(dst_root, "paper")

# 새로 생성할 book 폴더 경로
book_folder = os.path.join(dst_root, "book")
os.makedirs(book_folder, exist_ok=True)

# paper 폴더에서 파일들을 순회하며 인덱스가 R_3579 ~ R_3849인 파일들을 book 폴더로 이동
print("=== paper 폴더에서 book 파일 이동 시작 ===")
for fname in os.listdir(paper_folder):
    idx = extract_idx(fname)
    if idx is not None and 3579 <= idx <= 3849:
        src_path = os.path.join(paper_folder, fname)
        dst_path = os.path.join(book_folder, fname)
        shutil.move(src_path, dst_path)
        print(f"{fname} 이동 완료")
print("=== 이동 완료! ===")

## croma database 문서 입력 및 임베딩 후 호출

!pip install -U langchain-community
!pip install langchain chromadb unstructured sentence-transformers docarray

In [25]:
import os
os.makedirs("chroma_db", exist_ok=True)

# ---- corrected imports ----
from langchain.document_loaders import UnstructuredWordDocumentLoader, PyPDFLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import BaseRetriever   # ← moved here

def ingest_to_chromadb(
    source_path: str,
    persist_directory: str = "./chroma_db",
    collection_name: str = "waste_policy",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    embedding_model: str = "all-MiniLM-L6-v2"
) -> Chroma:
    if source_path.lower().endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(source_path)
    elif source_path.lower().endswith(".pdf"):
        loader = PyPDFLoader(source_path)
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. (.docx, .pdf 만 가능)")
    docs = loader.load()

    splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=["Heading 1","Heading 2","Heading 3"],
    max_chunk_size=chunk_size,    # chunk_size → max_chunk_size 로 변경
    chunk_overlap=chunk_overlap)   # 그대로 사용

    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

    vectordb = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    vectordb.persist()
    return vectordb

def get_simple_retriever(vectordb: Chroma, k: int = 4) -> BaseRetriever:
    return vectordb.as_retriever(
        search_type="similarity", search_kwargs={"k": k}
    )


## Kaggle E-Waste Image Dataset

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("akshat103/e-waste-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 11.8M/11.8M [00:01<00:00, 9.00MB/s]


Extracting files...
Path to dataset files: C:\Users\Administrator\.cache\kagglehub\datasets\akshat103\e-waste-image-dataset\versions\1


## 